In [1]:
import pandas as pd
import numpy as np

In [2]:
INPUT_FILE = '../dataset/flipkart_products_preprocessed_1.csv'

In [3]:
df = pd.read_csv(INPUT_FILE)

In [4]:
df = df.drop(columns='Unnamed: 0')

In [5]:
mobiles = df[df["level_2"] == "Mobiles & Accessories"]
others  = df[df["level_2"] != "Mobiles & Accessories"]

mobiles_sampled = mobiles.sample(20000, random_state=42)

In [6]:
df = pd.concat([mobiles_sampled, others], ignore_index=True)

In [7]:
df["title_len"] = df["title"].astype(str).str.len()

In [8]:
df = df[df["title_len"] <= 500].reset_index(drop=True)

In [9]:
df["desc_len"] = df["description"].astype(str).str.len()

In [10]:
df = df[df["desc_len"] <= 5700].reset_index(drop=True)

In [11]:
df.isnull().sum()

url                     0
title                   0
price                   0
discount                0
rating_value        20423
hierarchy               0
description             0
hierarchy_list          0
hierarchy_depth         0
level_1                 0
level_2              4676
level_3              4676
level_4              4874
level_5             10281
level_6             21566
level_7            157048
level_8            257710
title_len               0
desc_len                0
dtype: int64

In [12]:
df = df.drop(['level_1', 'level_7', 'level_8', 'title_len', 'desc_len'], axis=1)

In [13]:
df["level_2"] = df["level_2"].replace("", "unk").fillna("unk")
df["level_3"] = df["level_3"].replace("", "unk").fillna("unk")

df["level_4"] = df["level_4"].fillna("")
df["level_5"] = df["level_5"].fillna("")
df["level_6"] = df["level_6"].fillna("")

In [14]:
df["level_combined"] = (
    df["level_4"].astype(str) + " " +
    df["level_5"].astype(str) + " " +
    df["level_6"].astype(str)
).str.strip()

df["level_combined"] = df["level_combined"].replace("", "unk")

In [15]:
median_ratings = df.groupby("level_2")["rating_value"].median()

In [16]:
df["rating_value"] = df["rating_value"].fillna(
    df["level_2"].map(median_ratings)
)

In [17]:
df["rating_value"] = df["rating_value"].fillna(df["rating_value"].median())

In [18]:
df = df.drop(['level_4', 'level_5', 'level_6', 'hierarchy', 'hierarchy_list', 'hierarchy_depth', 'url'], axis=1)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260027 entries, 0 to 260026
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   title           260027 non-null  object 
 1   price           260027 non-null  float64
 2   discount        260027 non-null  float64
 3   rating_value    260027 non-null  float64
 4   description     260027 non-null  object 
 5   level_2         260027 non-null  object 
 6   level_3         260027 non-null  object 
 7   level_combined  260027 non-null  object 
dtypes: float64(3), object(5)
memory usage: 15.9+ MB


In [20]:
df.to_csv('../dataset/flipkart_products_preprocessed_2.csv')